### Gurobi Implementation

In [1]:
### imports
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from matplotlib import pyplot as plt

#### Import and prep input data

In [2]:
# regression results
regResults = pd.read_csv('regression_results.csv').drop(columns = ['Unnamed: 0'])
regResults.sort_values('p-value', ascending = True)

,feature,coefficient estimate,p-value
18,population_OtherRace,0.585061,0.000000e+00
14,population_White,0.268790,0.000000e+00
15,population_Black,0.233985,5.675460e-13
2,perc_prenat_1tri,0.073212,2.376876e-11
13,beds_per_1000,-0.110453,5.656041e-09
1,perc_csec,-0.110188,2.349227e-08
17,population_Asian,0.380202,1.047958e-05
10,hiv_tested,-0.043750,5.959245e-05
0,intercept,-28.916958,7.478606e-04
12,perc_obese,0.063540,2.134964e-03


In [41]:
# parameter Data
parametersData = pd.read_csv('parameters.csv').drop(columns = ['Unnamed: 0'])
parametersData.head()

,county_name,perc_csec,perc_prenat_1tri,birth_rate_15_19,birth_rate_20_24,birth_rate_25_29,birth_rate_30_34,birth_date_35_39,gono_per_100000,perc_smoker,hiv_tested,perc_no_healthins,perc_obese,beds_per_1000,population_White,population_Black,population_Native,population_Asian,population_OtherRace,ADI_STATERNK_INT_mean
0,Potter,35.9,77.8,27.7,145.7,135.1,81.7,29.4,75.71931,21.0,33.0,8.0,36.0,1.5,97.093197,0.341624,0.173809,0.419539,0.299670,8.222222
1,Wyoming,32.4,71.2,16.3,83.4,108.6,95.1,37.2,75.71931,24.0,45.0,11.0,29.0,0.4,93.101411,1.805894,0.169880,0.435778,0.324987,5.318182
2,Lehigh,29.9,73.9,19.8,70.9,104.0,105.2,49.3,102.80000,18.0,45.0,6.0,32.0,4.7,76.151936,7.268782,0.328852,3.331809,6.020613,4.940367
3,Indiana,24.0,65.7,9.5,41.8,128.1,98.0,38.7,34.70000,21.0,37.0,8.0,39.0,2.0,94.083800,2.260161,0.065117,0.928217,0.455821,7.158730
4,Schuylkill,32.1,67.6,20.9,88.7,114.6,90.6,36.6,34.50000,17.0,45.0,7.0,34.0,1.7,93.046113,2.989397,0.164864,0.421320,1.268186,7.559322


In [27]:
# decide program vars and filter parameter data and regression results
program_vars = ['perc_prenat_1tri', 'perc_csec', 'perc_smoker', 'perc_no_healthins']

# regression results for programs 
programWeights = regResults[regResults.feature.apply(lambda x: x in program_vars)].reset_index()
weightDict = {}
for var in program_vars:
    varWeight = programWeights.loc[np.where(programWeights.feature == var)[0][0], 'coefficient estimate']
    weightDict[var] = varWeight
    
print(weightDict)
    

# parameter rates for program vars 
programData = parametersData.copy()
programData = programData[program_vars]
#programData['gono_per_100000'] = programData['gono_per_100000']/100000
programData = programData.drop_duplicates().reset_index(drop = True)
print(programData.shape)
programData.head()

{'perc_prenat_1tri': 0.0732123029191958, 'perc_csec': -0.1101875123869329, 'perc_smoker': -0.0189638084573768, 'perc_no_healthins': -0.04380370470907}
(67, 4)


,perc_prenat_1tri,perc_csec,perc_smoker,perc_no_healthins
0,77.8,35.9,21.0,8.0
1,71.2,32.4,24.0,11.0
2,73.9,29.9,18.0,6.0
3,65.7,24.0,21.0,8.0
4,67.6,32.1,17.0,7.0


In [28]:
# get the non-program dot product

# nonprogram vars 
nonprogram_vars = [v for v in parametersData.columns.tolist() if v not in program_vars]
nonprogram_vars.remove('county_name')

# regression results for nonprogram vars 
nonprogramWeights = regResults[regResults.feature.apply(lambda x: x in nonprogram_vars)]
intercept = regResults[regResults.feature == 'intercept']['coefficient estimate'][0]

# parameter rates for non program vars 
nonprogramData = parametersData.copy()
nonprogramData = nonprogramData[nonprogram_vars]
nonprogramData.drop_duplicates(inplace = True)
nonprogramData.head()

nonprogramSum = []
for i in range(nonprogramData.shape[0]):
    countyNonProgramSum = nonprogramWeights.values[:,1].dot(nonprogramData.values[i,:])
    nonprogramSum.append(countyNonProgramSum+intercept)
    

len(nonprogramSum)



67

#### Define model parameters

In [29]:
# define decision variables 
num_programs = len(program_vars)
num_counties = programData.shape[0]
counties = range(num_counties)
programs = range(num_programs)

# parameters--rates of outcomes of interest per county
# perhaps formatted as a matrix of row x col = number of counties x number of programs
#rates = np.array([67,num_programs])
rates = programData.values

# total goals
total_goals = 3

# equity constraint per county
# equity_cap = XXX

# parameter weights (from regression output)
program_weights = weightDict
weight_direction = [1,-1,-1,-1] # (-1 if we want less of an outcome, 1 if we want more of an outcome, for each program)



#### Model setup

In [30]:
# initialize model
model = gp.Model()

# decision variables
X = model.addVars(counties, programs)

# objective function
model.setObjective(sum(sum(program_weights[program_vars[j]]*((1+(weight_direction[j]*X[i,j]))*rates[i,j]) for j in programs) + nonprogramSum[i] for i in counties))

model.modelSense = GRB.MAXIMIZE

# constraints
for i in counties:
    #model.addConstr(sum(program_weights[j]*rates[i,j] for j in programs) - 
    #   sum(program_weights[j]*((1+(weight_direction[j]*X[i,j]))*rates[i,j]) for j in programs) >= equity_cap[i]) # equity constraint
    
    model.addConstr(sum(X[i,j] for j in programs) <= 1) # program resource allocation constraint
    
    model.addConstr(sum(program_weights[program_vars[j]]*((1+(weight_direction[j]*X[i,j]))*rates[i,j]) for j in programs) + nonprogramSum[i] <= total_goals)
    model.addConstr(sum(program_weights[program_vars[j]]*((1+(weight_direction[j]*X[i,j]))*rates[i,j]) for j in programs) + nonprogramSum[i] >= 0)
    
    for j in programs: 
        model.addConstr(X[i,j] >= 0) # non-negativity
        model.addConstr(((1+(weight_direction[j]*X[i,j]))*rates[i,j]) >= 0) # ensure rates are between [0,1]
        model.addConstr(((1+(weight_direction[j]*X[i,j]))*rates[i,j]) <= 100) # ensure rates are between [0,100] since already as a percent

# optimize
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1005 rows, 268 columns and 1608 nonzeros
Model fingerprint: 0xa9cc0ee4
Coefficient statistics:
  Matrix range     [2e-01, 9e+01]
  Objective range  [2e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-03, 9e+01]
Presolve removed 873 rows and 4 columns
Presolve time: 0.03s
Presolved: 132 rows, 270 columns, 534 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2224679e+02   7.254941e+01   0.000000e+00      0s
     169    2.0100000e+02   0.000000e+00   0.000000e+00      0s

Solved in 169 iterations and 0.07 seconds (0.00 work units)
Optimal objective  2.010000000e+02


In [31]:
model.ObjVal

201.0

#### Analyze output

In [32]:
# print allocations for first 10 counties as a check
for i in range(10):
    print('\nCounty ', i, ' allocations:')
    for j in programs: 
        print('Program ', j, ': ', X[i,j].x)


County  0  allocations:
Program  0 :  0.0
Program  1 :  0.6834942054191996
Program  2 :  0.0
Program  3 :  0.3165057945808004

County  1  allocations:
Program  0 :  0.40449438202247184
Program  1 :  0.5425421623139162
Program  2 :  0.0
Program  3 :  0.0

County  2  allocations:
Program  0 :  0.3119239228314685
Program  1 :  0.0
Program  2 :  0.0
Program  3 :  0.6880760771685315

County  3  allocations:
Program  0 :  0.0
Program  1 :  0.7121484756635322
Program  2 :  0.0
Program  3 :  0.0

County  4  allocations:
Program  0 :  0.0
Program  1 :  0.8373340277618113
Program  2 :  0.0
Program  3 :  0.0

County  5  allocations:
Program  0 :  0.0
Program  1 :  0.7594905769062927
Program  2 :  0.0
Program  3 :  0.0

County  6  allocations:
Program  0 :  0.10236514709767523
Program  1 :  0.0
Program  2 :  0.0
Program  3 :  0.8976348529023247

County  7  allocations:
Program  0 :  0.0
Program  1 :  0.44569938604362647
Program  2 :  0.0
Program  3 :  0.5543006139563736

County  8  allocations:
P

In [33]:
# allocation when not forcing counties to use all resources (x[i,j] <= 1)
program_investment = np.zeros(num_programs)
for i in counties: 
    for j in programs:
        if X[i,j].x > 0: 
            program_investment[j] +=1
            
program_investment

array([23., 44., 14., 25.])

In [20]:
### results for 5 programs (with teen preg)

# allocation when not forcing counties to use all resources (x[i,j] <= 1)
# program_investment = np.zeros(num_programs)
# for i in counties: 
#     for j in programs:
#         if X[i,j].x > 0: 
#             program_investment[j] +=1
            
# program_investment

#array([ 0.,  5., 63.,  8., 16.])

In [21]:
### results for 5 programs (with teen preg)
 
# allocation when forcing counties to use all resources (x[i,j] == 1)
#program_investment = np.zeros(num_programs)
# for i in counties: 
#     for j in programs:
#         if X[i,j].x > 0: 
#             program_investment[j] +=1
            
# program_investment

# answer: array([44.,  5., 63.,  9., 16.])

In [22]:
# compare to (predicted) baseline
baselineSum = []
for i in range(parametersData.shape[0]):
    baselineSum_i = regResults.values[1:,1].dot(parametersData.values[i,1:])
    baselineSum.append(baselineSum_i+intercept)
    

sum(baselineSum)


64.31351634237541

In [42]:
# update parameter Data with program investment decisions
for i in range(len(parametersData)):
    for j in programs: 
        parametersData.loc[i, 'program_'+program_vars[j]+'_investment_pct'] = X[i,j].x
        if X[i,j].x > 0:
            parametersData.loc[i, 'whetherinvestment_in_prog_'+program_vars[j]] = 1
        else:
            parametersData.loc[i, 'whetherinvestment_in_prog_'+program_vars[j]] = 0

In [43]:
parametersData

,county_name,perc_csec,perc_prenat_1tri,birth_rate_15_19,birth_rate_20_24,birth_rate_25_29,birth_rate_30_34,birth_date_35_39,gono_per_100000,perc_smoker,...,population_OtherRace,ADI_STATERNK_INT_mean,program_perc_prenat_1tri_investment_pct,whetherinvestment_in_prog_perc_prenat_1tri,program_perc_csec_investment_pct,whetherinvestment_in_prog_perc_csec,program_perc_smoker_investment_pct,whetherinvestment_in_prog_perc_smoker,program_perc_no_healthins_investment_pct,whetherinvestment_in_prog_perc_no_healthins
0,Potter,35.9,77.8,27.7,145.7,135.1,81.7,29.4,75.71931,21.0,...,0.299670,8.222222,0.000000,0.0,0.683494,1.0,0.0,0.0,0.316506,1.0
1,Wyoming,32.4,71.2,16.3,83.4,108.6,95.1,37.2,75.71931,24.0,...,0.324987,5.318182,0.404494,1.0,0.542542,1.0,0.0,0.0,0.000000,0.0
2,Lehigh,29.9,73.9,19.8,70.9,104.0,105.2,49.3,102.80000,18.0,...,6.020613,4.940367,0.311924,1.0,0.000000,0.0,0.0,0.0,0.688076,1.0
3,Indiana,24.0,65.7,9.5,41.8,128.1,98.0,38.7,34.70000,21.0,...,0.455821,7.158730,0.000000,0.0,0.712148,1.0,0.0,0.0,0.000000,0.0
4,Schuylkill,32.1,67.6,20.9,88.7,114.6,90.6,36.6,34.50000,17.0,...,1.268186,7.559322,0.000000,0.0,0.837334,1.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Juniata,21.9,65.7,14.8,105.2,133.7,106.9,46.6,16.20000,19.0,...,0.648903,6.000000,0.213079,1.0,0.000000,0.0,0.0,0.0,0.786921,1.0
63,Delaware,31.6,69.6,10.7,42.9,87.8,125.6,68.6,151.00000,12.0,...,1.282972,4.318681,0.000000,0.0,0.617709,1.0,0.0,0.0,0.000000,0.0
64,Susquehanna,31.8,64.2,19.3,95.0,129.8,92.5,33.4,10.70000,19.0,...,0.283223,5.628571,0.000000,0.0,0.952756,1.0,0.0,0.0,0.000000,0.0
65,Clinton,27.3,68.8,15.2,50.3,137.2,91.0,37.6,25.80000,21.0,...,0.264598,6.285714,0.000000,0.0,0.662520,1.0,0.0,0.0,0.000000,0.0


In [46]:
investment_vars = ['program_'+var+'_investment_pct' for var in program_vars ]
investment_vars

['program_perc_prenat_1tri_investment_pct',
 'program_perc_csec_investment_pct',
 'program_perc_smoker_investment_pct',
 'program_perc_no_healthins_investment_pct']

In [23]:
program_vars

['perc_prenat_1tri', 'perc_csec', 'perc_smoker', 'perc_no_healthins']

In [47]:
# investment patterns given investment in program j
parametersData[parametersData.whetherinvestment_in_prog_perc_csec>0][investment_vars].describe()

,program_perc_prenat_1tri_investment_pct,program_perc_csec_investment_pct,program_perc_smoker_investment_pct,program_perc_no_healthins_investment_pct
count,44.000000,44.000000,44.000000,44.000000
mean,0.018075,0.683185,0.042204,0.127551
std,0.083802,0.160439,0.135844,0.216855
min,0.000000,0.297344,0.000000,0.000000
25%,0.000000,0.569266,0.000000,0.000000
50%,0.000000,0.697821,0.000000,0.000000
75%,0.000000,0.802985,0.000000,0.328296
max,0.404494,0.995467,0.539261,0.702656


In [48]:
parametersData[parametersData.whetherinvestment_in_prog_perc_no_healthins>0][investment_vars].describe()

,program_perc_prenat_1tri_investment_pct,program_perc_csec_investment_pct,program_perc_smoker_investment_pct,program_perc_no_healthins_investment_pct
count,25.000000,25.000000,25.0,25.000000
mean,0.105277,0.255511,0.0,0.639213
std,0.118338,0.279982,0.0,0.190666
min,0.000000,0.000000,0.0,0.316506
25%,0.000000,0.000000,0.0,0.440372
50%,0.062309,0.000000,0.0,0.697865
75%,0.218098,0.559628,0.0,0.781902
max,0.311924,0.683494,0.0,0.937691


In [49]:
parametersData[parametersData.whetherinvestment_in_prog_perc_prenat_1tri>0][investment_vars].describe()

,program_perc_prenat_1tri_investment_pct,program_perc_csec_investment_pct,program_perc_smoker_investment_pct,program_perc_no_healthins_investment_pct
count,23.000000,23.000000,23.000000,23.000000
mean,0.213476,0.041931,0.283360,0.450786
std,0.108645,0.140112,0.402613,0.408743
min,0.007173,0.000000,0.000000,0.000000
25%,0.127645,0.000000,0.000000,0.000000
50%,0.222539,0.000000,0.000000,0.697865
75%,0.296955,0.000000,0.702622,0.784412
max,0.404494,0.542542,0.992827,0.937691


In [50]:
parametersData[parametersData.whetherinvestment_in_prog_perc_smoker>0][investment_vars].describe()

,program_perc_prenat_1tri_investment_pct,program_perc_csec_investment_pct,program_perc_smoker_investment_pct,program_perc_no_healthins_investment_pct
count,14.000000,14.000000,14.000000,14.0
mean,0.105909,0.153073,0.644474,0.0
std,0.130465,0.252660,0.258000,0.0
min,0.000000,0.000000,0.019853,0.0
25%,0.000000,0.000000,0.492742,0.0
50%,0.033582,0.000000,0.687715,0.0
75%,0.222318,0.345554,0.813430,0.0
max,0.321588,0.587464,0.992827,0.0


In [52]:
parametersData.to_csv('county_w_rates_and_priorities.csv')

In [56]:
parametersData.columns

Index(['county_name', 'perc_csec', 'perc_prenat_1tri', 'birth_rate_15_19',
       'birth_rate_20_24', 'birth_rate_25_29', 'birth_rate_30_34',
       'birth_date_35_39', 'gono_per_100000', 'perc_smoker', 'hiv_tested',
       'perc_no_healthins', 'perc_obese', 'beds_per_1000', 'population_White',
       'population_Black', 'population_Native', 'population_Asian',
       'population_OtherRace', 'ADI_STATERNK_INT_mean',
       'program_perc_prenat_1tri_investment_pct',
       'whetherinvestment_in_prog_perc_prenat_1tri',
       'program_perc_csec_investment_pct',
       'whetherinvestment_in_prog_perc_csec',
       'program_perc_smoker_investment_pct',
       'whetherinvestment_in_prog_perc_smoker',
       'program_perc_no_healthins_investment_pct',
       'whetherinvestment_in_prog_perc_no_healthins'],
      dtype='object')